## Домашнее задание

Попробуйте сделать функцию-враппер (функцию-обертку) вокруг DESeq2, которая будет работать из-под Python и выполнять дифференциальную экспрессию по матрице каунтов, а также по датасету с описанием образцов, которые будут даваться на вход функции. Продумайте, как лучше реализовать хранение аутпута.

Приспособьте эту функцию для scanpy так, чтобы она могла определять дифференциально экспрессированные гены для кластеров, используя их псевдобалки по отдельным образцам. В каком случае имеет смысл использовать эту функцию?

Добработайте функцию и напишите новую функцию на основе `sc.tl.rank_genes_groups()`, чтобы она была аналогом функции <a href="https://satijalab.org/seurat/reference/findconservedmarkers"><code>FindConservedMarkers()</code></a> из Seurat.

## Решение

### Установка пакетов и импорт данных

Установим необходимые библиотеки

In [ ]:
!pip3 install scanpy[leiden] anndata2ri scikit-misc scvi-tools opentsne harmonypy

# Необходимо, чтобы конкретно прогрузился matplotlib
import os
os.kill(os.getpid(), 9)

      Successfully uninstalled matplotlib-3.2.2


Обновим версию gdown

In [ ]:
!pip install --upgrade --no-cache-dir gdown

Импортируем необходимые модули в python

In [ ]:
!pip install rpy2==3.5.1

In [3]:
import rpy2

rpy2.__version__

'3.5.1'

In [13]:
import scanpy as sc

import anndata2ri
anndata2ri.activate()

%reload_ext rpy2.ipython

import rpy2.robjects as ro
ro.numpy2ri.activate()

from rpy2.robjects import pandas2ri, Formula
from rpy2.robjects.packages import importr

sc.set_figure_params(dpi=150)
sc.settings.verbosity = 3

In [83]:
!apt-get install libgmp3-dev libcairo2-dev libharfbuzz-dev libfribidi-dev libudunits2-dev
!gdown --id 1W0TuuQospxcasOX5cQdhT_Xd-k7-QAck
!tar xf library.tar.gz
!cp -r /content/usr/local/lib/R /usr/local/lib/ >> None
!rm -r usr library.tar.gz

/usr/local/lib/python3.8/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1W0TuuQospxcasOX5cQdhT_Xd-k7-QAck
To: /content/library.tar.gz
100% 2.33G/2.33G [00:19<00:00, 120MB/s]
tar: Removing leading `/' from member names


In [ ]:
%R .libPaths("usr/local/lib/R/site-library/")

!Rscript -e 'install.packages("dplyr")'
!Rscript -e 'install.packages("ggplot2")'
!Rscript -e 'install.packages("tidyr")'
!Rscript -e 'install.packages("stringr")'
!Rscript -e 'install.packages("statmod")'
!Rscript -e 'install.packages("MASS")'
!Rscript -e 'BiocManager::install("EnhancedVolcano")'
!Rscript -e 'BiocManager::install("DESeq2")'
!Rscript -e 'BiocManager::install("apeglm")'
!Rscript -e 'install.packages("repr")'
!Rscript -e 'BiocManager::install("glmGamPoi")'

In [14]:
%%R

suppressPackageStartupMessages({
  library(edgeR)
  library(stringr)
  library(dplyr)
  library(tidyr)
  library(ggplot2)
  library(MASS)
  library(glue)
  library(EnhancedVolcano)
  library(DESeq2)
  library(apeglm)
  library(repr)
  library(glmGamPoi)
})

Скачаем single cell данные

In [5]:
!gdown "1jW548g6ERFS0t7NywgyjRs6VaE5QwXbg&confirm=t"

Downloading...
From: https://drive.google.com/uc?id=1jW548g6ERFS0t7NywgyjRs6VaE5QwXbg&confirm=t
To: /content/PBMC_Satija.h5ad
100% 243M/243M [00:02<00:00, 82.6MB/s]


Прочитаем `.h5ad` файл

In [58]:
adata_raw = sc.read_h5ad("PBMC_Satija.h5ad")
adata_raw.layers["counts"] = adata_raw.X.copy()

Выберем из изначального датасета только 20 клеток. Это необходимо, так как при анализе дифф. экспрессии всего датасета переполняется ОЗУ и колаб автоматически сбрасывает все локальные переменные.

In [60]:
adata = adata_raw.copy()
adata = adata[:20, :]

Пусть тестируемыми группами будут колонки: батчи (образцы тканей от различных доноров) и типу клеток, к которому принадлежит та или иная клетка из scRNA-seq.

In [61]:
adata.obs['donor'].value_counts()

P2    9
P1    8
P3    3
Name: donor, dtype: int64

In [62]:
adata.obs['celltype.l1'].value_counts()

CD8 T      8
CD4 T      7
NK         2
B          1
Mono       1
other T    1
Name: celltype.l1, dtype: int64

In [65]:
adata.obs.head()

,orig.ident,lane,donor,time,celltype.l1,celltype.l2,celltype.l3,Phase
L1_AAACCCACATCAGTCA,P3_3,L1,P3,3,CD8 T,CD8 TEM,CD8 TEM_1,G1
L1_AAACCCATCCACACCT,P3_3,L1,P3,3,CD8 T,CD8 Naive,CD8 Naive,S
L1_AAACCCATCTGCGGAC,P1_0,L1,P1,0,CD4 T,CD4 TCM,CD4 TCM_1,S
L1_AAACCCATCTTAAGGC,P2_3,L1,P2,3,CD4 T,CD4 CTL,CD4 CTL,S
L1_AAACGAAAGATAACAC,P2_3,L1,P2,3,B,B naive,B naive kappa,S


### DESeq2 с single cell данными

Напишем функцию-оболочку DESeq2, которая будет проводить анализ дифф. экспрессии c AnnData объектом. 

Вместо указания в аргументе функциии описания образцов добавил аргумент `formula`, который позволяет указывать на вход функции, какой будет дизайн у эксперимента.

In [142]:
def deseq2(adata, formula, output_name='deseq2_output'):
    '''
    Wrapper function of DESeq2 R package, that compute differential expression by condition (formula).
    ----------
    adata : AnnData
        Annotated data matrix.
    formula : string
        Formula for DESeq2 model
    '''

    # Import DESeq2 library from R to Python
    deseq2 = importr("DESeq2")

    # Create DESeqDataSet from AnnData object
    dds = deseq2.DESeqDataSetFromMatrix(countData=adata.X.T, 
                                        colData=adata.obs,
                                        design=Formula(formula))
    
    # Evaluate differential expression by DESeq2 model
    dds = deseq2.DESeq(dds)

    # Write DESeq2 results into AnnData object    
    adata.uns["deseq2"] = dds

    # Save DESeq2 output
    save_deseq2_results(adata, output_name)

Сохранять DESeq2 результаты будем в виде двух файлов:
1. `.rds` файл — в нем будет находиться `DESeqDataSet` объект с результатами DESeq2
2. `.h5ad` файл — в нем будет сохранен изначальный `AnnData` объект.

Выбрал такую логику сохранения, так как это позволяет работать с результатами дифф. экспрессии прямо в Python.

In [132]:
def save_deseq2_results(adata, output_name):
    '''
    This saves AnnData Object with DESeqDataSet object into two file:
    .rds file (DESeqDataSet object) and
    .h5ad file (AnnData object). 
    ----------
    adata : AnnData
        Annotated data matrix.
    file_name : string
        Save file name.
    '''

    # import saveRDS R library into Python
    saveRDS = ro.r['saveRDS']

    # Save R DESeqDataSet object with DESeq2 results into .rds file
    saveRDS(adata.uns["deseq2"], f'{output_name}.rds')

    del adata.uns["deseq2"]

    # Save Python object AnnData object without DESeq2 results into .h5ad file
    adata.write(f'{output_name}.h5ad')

In [133]:
formula = '~ donor'

In [ ]:
deseq2(adata, formula)

Напишем функцию для чтения результатов DESeq2.

In [85]:
def read_deseq2_output(output_name):
    '''
    This reads DESeqDataSet object and AnnData object and returns AnnData Object with DESeq2 results.
    ----------
    adata : AnnData
        Annotated data matrix.
    file_name : string
        Save file name.
    '''

    # Read .h5ad file with AnnData object
    adata = sc.read(f'{output_name}.h5ad')

    # Import readRDS library
    readRDS = ro.r['readRDS']

    # Read .rds file and add it into AnnData object
    adata.uns["deseq2"] = readRDS(f'{output_name}.rds')

    return adata

In [135]:
adata_with_deseq2 = read_deseq2_output('deseq2_output')

In [140]:
dds = adata_with_deseq2.uns['deseq2']

In [143]:
%%R -i dds

dds <- dds[rowSums(counts(dds, normalized = TRUE)) >= 100,]
head(results(dds))

log2 fold change (MLE): donor P3 vs P1 
Wald test p-value: donor P3 vs P1 
DataFrame with 6 rows and 6 columns
   baseMean log2FoldChange     lfcSE      stat    pvalue      padj
  <numeric>      <numeric> <numeric> <numeric> <numeric> <numeric>
1  12.03048      -0.285783  0.412257 -0.693215  0.488174  0.834904
2  33.65125      -0.176118  0.255475 -0.689376  0.490587  0.834904
3   6.50988       0.336214  0.680931  0.493756  0.621479  0.894489
4   7.30482       0.258107  0.531374  0.485736  0.627154  0.894489
5  25.71418      -0.129026  0.302716 -0.426229  0.669941  0.894489
6  15.91041      -0.321414  0.362301 -0.887147  0.375000  0.792609
